In [1]:
! pip install lightgbm numpy pandas rdkit scikit-learn

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 3.1 MB 379 kB/s eta 0:00:01
     |████████████████████████████████| 34.4 MB 28.8 MB/s eta 0:00:01    |███                             | 3.3 MB 34.9 MB/s eta 0:00:01


In [2]:
import lightgbm as lgb
import numpy as np
import pandas as pd

from rdkit import Chem
from rdkit.Chem.rdMolDescriptors import GetMorganFingerprintAsBitVect
from sklearn.model_selection import train_test_split
from sklearn.metrics import fbeta_score

In [3]:
DATA_PATH = "/bohr/AI4S-dwdx/v1/AI4science/"

raw_data = pd.read_csv(f"{DATA_PATH}/mol_train.csv")
test_data = pd.read_csv(f"{DATA_PATH}/mol_test.csv")

rd = 2024
train_data, valid_data = train_test_split(
    raw_data, test_size=0.2, random_state=rd)

In [4]:
from rdkit.Chem import Descriptors
def calculate_1dqsar_repr(smiles):
    mol = Chem.MolFromSmiles(smiles)
    mol_weight = Descriptors.MolWt(mol)  
    log_p = Descriptors.MolLogP(mol)  
    num_h_donors = Descriptors.NumHDonors(mol)  
    num_h_acceptors = Descriptors.NumHAcceptors(mol)
    tpsa = Descriptors.TPSA(mol)
    num_rotatable_bonds = Descriptors.NumRotatableBonds(mol)
    num_aromatic_rings = Descriptors.NumAromaticRings(mol)
    num_aliphatic_rings = Descriptors.NumAliphaticRings(mol)
    num_saturated_rings = Descriptors.NumSaturatedRings(mol)
    num_heteroatoms = Descriptors.NumHeteroatoms(mol)
    num_valence_electrons = Descriptors.NumValenceElectrons(mol)
    num_radical_electrons = Descriptors.NumRadicalElectrons(mol) 
    num_polar_hydrogens = Descriptors.NumHAcceptors(mol)
 
    count_n = 0
    for atom in mol.GetAtoms():
        atomic_num = atom.GetAtomicNum()
        if atomic_num == 7: 
            count_n += 1


    features_repr = [mol_weight, log_p, num_h_donors, num_h_acceptors, tpsa, num_rotatable_bonds,
                     num_aromatic_rings, num_aliphatic_rings, num_saturated_rings, num_heteroatoms,
                     num_valence_electrons, num_radical_electrons, count_n, num_polar_hydrogens]
    return features_repr

train_data["1dqsar_mr"] = train_data["SMILES"].apply(calculate_1dqsar_repr)
valid_data["1dqsar_mr"] = valid_data["SMILES"].apply(calculate_1dqsar_repr)


/tmp/ipykernel_826/281782258.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data["1dqsar_mr"] = train_data["SMILES"].apply(calculate_1dqsar_repr)
/tmp/ipykernel_826/281782258.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_data["1dqsar_mr"] = valid_data["SMILES"].apply(calculate_1dqsar_repr)


In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
import lightgbm as lgb
from sklearn.svm import SVC
import matplotlib.pyplot as plt


train_x = np.array(train_data["1dqsar_mr"].values.tolist())
train_y = np.array(train_data["TARGET"].values.tolist())
valid_x = np.array(valid_data["1dqsar_mr"].values.tolist())
valid_y = np.array(valid_data["TARGET"].values.tolist())

scaler = StandardScaler()
train_x = scaler.fit_transform(train_x)
valid_x = scaler.transform(valid_x)

classifiers = [
    ("Logistic Regression", LogisticRegression(random_state=rd)),
    ("K-Nearest Neighbors", KNeighborsClassifier()),
    ("Random Forest", RandomForestClassifier(random_state=rd)),
    ("LightGBM", lgb.LGBMClassifier(verbose=-1)),
    ("SVM", SVC(kernel='linear'))
]
# ("XGBoost", XGBClassifier(use_label_encoder=False, eval_metric="logloss", random_state=rd)),


In [6]:
results = {}
threshold = 0.5
for name, classifier in classifiers:
    classifier.fit(train_x, train_y)
    valid_pred = classifier.predict(valid_x)
    # pred_y_proba = classifier.predict_proba(valid_x)[:, 1]
    valid_result = [1 if x > threshold else 0 for x in valid_pred]
    valid_score = fbeta_score(valid_data["TARGET"], valid_result, beta=2)
    print(f"[1D-QSAR][{name}]\tValid Score: {valid_score}")

[1D-QSAR][Logistic Regression]	Valid Score: 0.782442748091603
[1D-QSAR][K-Nearest Neighbors]	Valid Score: 0.7058823529411764
[1D-QSAR][Random Forest]	Valid Score: 0.7558139534883721
[1D-QSAR][LightGBM]	Valid Score: 0.7061068702290076
[1D-QSAR][SVM]	Valid Score: 0.7662835249042147


In [7]:
from rdkit.Chem import AllChem

def calculate_2dqsar_repr(smiles):    
    mol = Chem.MolFromSmiles(smiles)
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, 4, nBits=1024)
    return np.array(fp)

train_data["2dqsar_mr"] = train_data["SMILES"].apply(calculate_2dqsar_repr) 
valid_data["2dqsar_mr"] = valid_data["SMILES"].apply(calculate_2dqsar_repr) 

/tmp/ipykernel_826/2662707181.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data["2dqsar_mr"] = train_data["SMILES"].apply(calculate_2dqsar_repr)
/tmp/ipykernel_826/2662707181.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_data["2dqsar_mr"] = valid_data["SMILES"].apply(calculate_2dqsar_repr)


In [8]:
train_x = np.array(train_data["2dqsar_mr"].values.tolist())
# train_y = np.array(train_data["TARGET"].values.tolist())
valid_x = np.array(valid_data["2dqsar_mr"].values.tolist())
# valid_y = np.array(valid_data["TARGET"].values.tolist())


In [9]:
# results_2D = {}
threshold = 0.5
for name, classifier in classifiers:
    classifier.fit(train_x, train_y)
    valid_pred = classifier.predict(valid_x)
    valid_result = [1 if x > threshold else 0 for x in valid_pred]
    valid_score = fbeta_score(valid_data["TARGET"], valid_result, beta=2)
    print(f"[2D-QSAR][{name}]\tValid Score: {valid_score}")

[2D-QSAR][Logistic Regression]	Valid Score: 0.7226562500000001
[2D-QSAR][K-Nearest Neighbors]	Valid Score: 0.6704980842911877
[2D-QSAR][Random Forest]	Valid Score: 0.7
[2D-QSAR][LightGBM]	Valid Score: 0.7945736434108527
[2D-QSAR][SVM]	Valid Score: 0.6976744186046512


In [10]:
from rdkit.Chem import rdPartialCharges

def calculate_3dqsar_repr(SMILES, max_atoms=100, three_d=False):
    mol = Chem.MolFromSmiles(SMILES) 
    mol = Chem.AddHs(mol) 
    if three_d:
        AllChem.EmbedMolecule(mol, AllChem.ETKDG()) 
    else:
        AllChem.Compute2DCoords(mol)  
    natoms = mol.GetNumAtoms()  
    rdPartialCharges.ComputeGasteigerCharges(mol)  
    charges = np.array([float(atom.GetProp("_GasteigerCharge")) for atom in mol.GetAtoms()]) 
    coords = mol.GetConformer().GetPositions() 
    coulomb_matrix = np.zeros((max_atoms, max_atoms))  
    n = min(max_atoms, natoms)
    for i in range(n):
        for j in range(i, n):
            if i == j:
                coulomb_matrix[i, j] = 0.5 * charges[i] ** 2
            if i != j:
                delta = np.linalg.norm(coords[i] - coords[j]) 
                if delta != 0:
                    coulomb_matrix[i, j] = charges[i] * charges[j] / delta  
                    coulomb_matrix[j, i] = coulomb_matrix[i, j]
    coulomb_matrix = np.where(np.isinf(coulomb_matrix), 0, coulomb_matrix)  
    coulomb_matrix = np.where(np.isnan(coulomb_matrix), 0, coulomb_matrix)  
    return coulomb_matrix.reshape(max_atoms*max_atoms).tolist()


train_data["3dqsar_mr"] = train_data["SMILES"].apply(calculate_3dqsar_repr) 
valid_data["3dqsar_mr"] = valid_data["SMILES"].apply(calculate_3dqsar_repr) 

/tmp/ipykernel_826/635456171.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data["3dqsar_mr"] = train_data["SMILES"].apply(calculate_3dqsar_repr)
/tmp/ipykernel_826/635456171.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_data["3dqsar_mr"] = valid_data["SMILES"].apply(calculate_3dqsar_repr)


In [11]:
train_x = np.array(train_data["3dqsar_mr"].values.tolist())
# train_y = np.array(train_data["TARGET"].values.tolist())
valid_x = np.array(valid_data["3dqsar_mr"].values.tolist())
# valid_y = np.array(valid_data["TARGET"].values.tolist())

threshold = 0.5
for name, classifier in classifiers:
    classifier.fit(train_x, train_y)
    valid_pred = classifier.predict(valid_x)
    valid_result = [1 if x > threshold else 0 for x in valid_pred]
    valid_score = fbeta_score(valid_data["TARGET"], valid_result, beta=2)
    print(f"[3D-QSAR][{name}]\tValid Score: {valid_score}")

[3D-QSAR][Logistic Regression]	Valid Score: 0.0
[3D-QSAR][K-Nearest Neighbors]	Valid Score: 0.8069620253164558
[3D-QSAR][Random Forest]	Valid Score: 0.5394190871369293
[3D-QSAR][LightGBM]	Valid Score: 0.6626506024096385
[3D-QSAR][SVM]	Valid Score: 0.0


In [12]:
from sklearn.decomposition import PCA

scaler = StandardScaler()
train_x_scaled = scaler.fit_transform(train_x)
valid_x_scaled = scaler.fit_transform(valid_x)

pca = PCA(n_components=50)
train_x_pca = pca.fit_transform(train_x_scaled)
valid_x_pca = pca.fit_transform(valid_x_scaled)


threshold = 0.5
for name, classifier in classifiers:
    classifier.fit(train_x_pca, train_y)
    valid_pred = classifier.predict(valid_x_pca)
    valid_result = [1 if x > threshold else 0 for x in valid_pred]
    valid_score = fbeta_score(valid_data["TARGET"], valid_result, beta=2)
    print(f"[3D-QSAR][{name}]\tValid Score: {valid_score}")


/opt/conda/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[3D-QSAR][Logistic Regression]	Valid Score: 0.7230769230769231
[3D-QSAR][K-Nearest Neighbors]	Valid Score: 0.5514705882352942
[3D-QSAR][Random Forest]	Valid Score: 0.0
[3D-QSAR][LightGBM]	Valid Score: 0.0
[3D-QSAR][SVM]	Valid Score: 0.7275541795665635


In [13]:
from sklearn.decomposition import PCA

scaler = StandardScaler()
train_x_scaled = scaler.fit_transform(train_x)
valid_x_scaled = scaler.fit_transform(valid_x)

pca = PCA(n_components=0.85)
pca.fit(train_x_scaled)

train_x_pca = pca.transform(train_x_scaled)
valid_x_pca = pca.transform(valid_x_scaled)


threshold = 0.5
for name, classifier in classifiers:
    classifier.fit(train_x_pca, train_y)
    valid_pred = classifier.predict(valid_x_pca)
    valid_result = [1 if x > threshold else 0 for x in valid_pred]
    valid_score = fbeta_score(valid_data["TARGET"], valid_result, beta=2)
    print(f"[QSAR-fusion][{name}]\tValid Score: {valid_score}")


/opt/conda/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[QSAR-fusion][Logistic Regression]	Valid Score: 0.0
[QSAR-fusion][K-Nearest Neighbors]	Valid Score: 0.6727272727272726
[QSAR-fusion][Random Forest]	Valid Score: 0.0
[QSAR-fusion][LightGBM]	Valid Score: 0.04651162790697674
[QSAR-fusion][SVM]	Valid Score: 0.0


In [14]:
X_1dqsar = np.stack(train_data["1dqsar_mr"].values)
X_2dqsar = np.stack(train_data["2dqsar_mr"].values)
X_3dqsar = np.stack(train_data["3dqsar_mr"].values)

train_X = np.hstack((X_1dqsar,X_2dqsar,X_3dqsar))
scaler = StandardScaler()
train_X_scaled = scaler.fit_transform(train_X)

# len(train_X_scaled[0])


X_1dqsar_v = np.stack(valid_data["1dqsar_mr"].values)
X_2dqsar_v = np.stack(valid_data["2dqsar_mr"].values)
X_3dqsar_v = np.stack(valid_data["3dqsar_mr"].values)
valid_X = np.hstack((X_1dqsar_v,X_2dqsar_v,X_3dqsar_v))
scaler = StandardScaler()
valid_X_scaled = scaler.fit_transform(valid_X)

#len(valid_X_scaled[0])



In [15]:
from sklearn.feature_selection import SelectFromModel

selector = SelectFromModel(RandomForestClassifier(n_estimators=100, random_state=rd))
selector.fit_transform(train_X_scaled, train_y)

train_X_selected = selector.transform(train_X_scaled)
valid_X_selected = selector.transform(valid_X_scaled) 

pca = PCA(n_components=0.5)
pca.fit(train_X_selected) 
train_X_pca = pca.transform(train_X_selected)
valid_X_pca = pca.transform(valid_X_selected) 

print(train_X_selected.shape)
print(valid_X_selected.shape)

(560, 1876)
(140, 1876)


In [16]:
threshold = 0.5
for name, classifier in classifiers:
    classifier.fit(train_X_pca, train_y)
    valid_pred = classifier.predict(valid_X_pca)
    valid_result = [1 if x > threshold else 0 for x in valid_pred]
    valid_score = fbeta_score(valid_data["TARGET"], valid_result, beta=2)
    print(f"[QSAR-fusion][{name}]\tValid Score: {valid_score}")

[QSAR-fusion][Logistic Regression]	Valid Score: 0.36480686695278963
[QSAR-fusion][K-Nearest Neighbors]	Valid Score: 0.6690140845070423
[QSAR-fusion][Random Forest]	Valid Score: 0.38626609442060084
[QSAR-fusion][LightGBM]	Valid Score: 0.46218487394957986
[QSAR-fusion][SVM]	Valid Score: 0.3813559322033898


In [17]:
test_data["1dqsar_mr"] = test_data["SMILES"].apply(calculate_1dqsar_repr)
test_data["2dqsar_mr"] = test_data["SMILES"].apply(calculate_2dqsar_repr)
test_data["3dqsar_mr"] = test_data["SMILES"].apply(calculate_3dqsar_repr)

X_1dqsar_test = np.stack(test_data["1dqsar_mr"].values)
X_2dqsar_test = np.stack(test_data["2dqsar_mr"].values)
X_3dqsar_test = np.stack(test_data["3dqsar_mr"].values)

In [18]:
test_X = np.hstack((X_1dqsar_test,X_2dqsar_test,X_3dqsar_test))
scaler = StandardScaler()
test_X_scaled = scaler.fit_transform(test_X)

In [19]:
selector = SelectFromModel(RandomForestClassifier(n_estimators=100, random_state=rd))
selector.fit_transform(train_X_scaled, train_y)

train_X_selected = selector.transform(train_X_scaled)
test_X_selected = selector.transform(test_X_scaled) 

pca = PCA(n_components=0.5)
pca.fit(train_X_selected) 
train_X_pca = pca.transform(train_X_selected)
test_X_pca = pca.transform(test_X_selected) 


In [20]:
model = KNeighborsClassifier()
model.fit(train_X_pca, train_y)
pred = model.predict(test_X_pca)
threshold = 0.5
result = [1 if x > threshold else 0 for x in pred]

In [21]:
sum(result)/len(result)

0.4822888283378747

In [22]:
submission = pd.DataFrame()  
submission["SMILES"] = test_data["SMILES"] 
submission["TARGET"] = result 
submission.to_csv("./submission.csv", index=False) 